In [1]:
!pip install pytorch-tabnet
!pip install wget
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path
import shutil
import gzip

from matplotlib import pyplot as plt
%matplotlib inline

Заявлено 99.2.

In [2]:
data = pd.read_csv('train.csv')

In [3]:
y = data['hand']
df = data.drop(['hand'], axis=1)

In [4]:
df.isnull().mean()

S1    0.0
C1    0.0
S2    0.0
C2    0.0
S3    0.0
C3    0.0
S4    0.0
C4    0.0
S5    0.0
C5    0.0
dtype: float64

In [8]:
categorical_columns = []
categorical_dims =  {}
for col in df.columns:
    categorical_columns.append(col)
    categorical_dims[col] = len(np.unique((df[col].values)))


In [9]:
features = [ col for col in df.columns] 

cat_idxs = [ i for i, f in enumerate(features)]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features)]


In [10]:
y.value_counts()

0    12493
1    10599
2     1206
3      513
4       93
5       54
6       36
7        6
9        5
8        5
Name: hand, dtype: int64

In [11]:
clf = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=1,
    lambda_sparse=1e-4, momentum=0.7, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15
)

Device used : cpu


In [26]:
cat_dims

[4, 13, 4, 13, 4, 13, 4, 13, 4, 13]

In [19]:
X_train_scale.shape

(20008, 10)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

In [14]:
scaler = MinMaxScaler()
X_train = X_train.values
X_test =  X_test.values

X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [22]:
X_train_scale.shape

(20008, 10)

In [15]:
max_epochs = 250 

In [24]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'test'],
    max_epochs=max_epochs, patience=100,
    batch_size=512, virtual_batch_size=256
) 

IndexError: ignored